In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import import_ipynb
import WikiCrawler as wk
import DataCleaner as dc
import pandas as pd
from bs4 import BeautifulSoup

importing Jupyter notebook from WikiCrawler.ipynb
importing Jupyter notebook from DataCleaner.ipynb


In [2]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
analyzer = SentimentIntensityAnalyzer()
url = "https://en.wikipedia.org/wiki/2019"

SentimentAnalyzerScore returns the sentiment score of the text.

In [4]:
def sentimentAnalyzerScore(text):
    score = analyzer.polarity_scores(text)
    return score

textMiner extracts the title and context from the Wikipedia page and stores it in a dataframe.

In [5]:
def textMiner(url):
    response = wk.initResp(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = wk.titleExtractor(soup)
    context = wk.contextExtractor(soup)
    wikiPageData = pd.DataFrame([[title,context]])
    dc.lowerOnly(wikiPageData)
    dc.alphaCharOnly(wikiPageData)
    return wikiPageData

stopWordDel removes all the stopwords from a given sentence and returns a filtered sentence.

In [6]:
def stopWordDel(sentence):
    stop_words = set(stopwords.words('english'))
    filteredSent = " "
    pageWord = sentence.split()
    for r in pageWord:
        if not r in stop_words:
            filteredSent = filteredSent + " " + r
    return filteredSent


getPageSentiment combines all three functions and returns the sentiment score of the passed Wikipedia page url.

In [7]:
def getPageSentiment(url):
    df = textMiner(url)
    text = stopWordDel(df.loc[0,1])
    score = sentimentAnalyzerScore(text)
    return score

In [18]:
print(getPageSentiment(url = "https://en.wikipedia.org/wiki/Black_Death") , 
      " title: ", textMiner("https://en.wikipedia.org/wiki/Black_Death").iloc[0,0])

print(getPageSentiment(url = "https://en.wikipedia.org/wiki/Independence_Day_(India)") , 
      " title: ", textMiner("https://en.wikipedia.org/wiki/Independence_Day_(India)").iloc[0,0])

print(getPageSentiment(url = "https://en.wikipedia.org/wiki/Mona_Lisa") , 
      " title: ", textMiner("https://en.wikipedia.org/wiki/Mona_Lisa").iloc[0,0])


{'neg': 0.158, 'neu': 0.779, 'pos': 0.062, 'compound': -0.9999}  title:    black death wikipedia
{'neg': 0.084, 'neu': 0.828, 'pos': 0.088, 'compound': -0.1082}  title:    independence day india wikipedia
{'neg': 0.057, 'neu': 0.848, 'pos': 0.095, 'compound': 0.9988}  title:    mona lisa wikipedia
